# Radhika Mardikar, Xinxin Mo

## Step 1:
Looking at the KEGG pathway, we will select 4 enzymes from the glycolysis, TCA cycle and pentose phosphate cycle. 
Glycolysis: hexokinase 1, phosphoglucose isomerase, phosphofructokinase, fructose-bisphosphate aldolase (https://www.ebi.ac.uk/interpro/potm/2004_2/Page2.htm)
TCA: citrate synthase, aconitase, isocitrase dehydrogenase, alpha-ketoglurate (https://www.news-medical.net/life-sciences/Krebs-Cycle-Enzymes.aspx)
pentose phosphate: transketolase, transaldolase, lactonase, phosphopentose isomerase (https://mcb.berkeley.edu/labs/krantz/mcb102/lect_S2008/MCB102-SPRING2008-LECTURE5-PENTOSE.pdf)

In [48]:
# import packages
from Bio import Entrez
from Bio import SeqIO
import prettytable as pt
import sqlite3

In [49]:
Entrez.email = "rmardikar@berkeley.edu"
# first row is glycolysis enzyme, second is pentose phosphate cycle, third is TCA
enzymelist = ["2.7.1.1", "5.4.2.2", "3.1.3.11", "5.3.1.9",
              "5.3.1.9", "3.1.3.11", "1.1.1.49", "1.1.1.44",
              "1.1.1.37", "1.2.4.1", "1.1.1.27", "1.8.1.4"]
# "1.1.1.343"
# 
organismlist = ['Homo sapiens', 'Drosophila melanogaster', "Escherichia coli"]
idlist, idlist2 = [], []

for org in organismlist:
    for enzyme in enzymelist:
        handle = Entrez.esearch(db="gene",
                                term = org + '[ORGN] AND ' + enzyme, 
                                idtype = 'acc', 
                                sort='relevance',
                                retmax=1)
        #print(org + '[ORGN]' + enzyme)
        record = Entrez.read(handle)
        idlist.append(record["IdList"])
        
        handle2 = Entrez.esearch(db="nucleotide",
                                term = org + '[ORGN] AND ' + enzyme, 
                                idtype = 'acc', 
                                sort='relevance',
                                retmax=1)
        record2 = Entrez.read(handle2)
        idlist2.append(record2["IdList"])


In [43]:
# create a database
# connect to database
conn = sqlite3.connect('my.db')
c = conn.cursor()

"""
path1 = enzymelist[0:3]
path2 = enzymelist[4:7]
path3 = enzymelist[8:11]
"""
# create pathway table
c.execute('CREATE TABLE pathway (name TEXT, description TEXT);')
# create enzymes table
c.execute('CREATE TABLE enzymes (name TEXT, enzyme_number TEXT, pathway_name TEXT);')
# create genes table
c.execute('CREATE TABLE genes (iD TEXT, name TEXT, organism TEXT, chromosome TEXT, start INT, end INT, sequence TEXT);')

# insert values into pathway table
c.executemany('INSERT INTO pathway(name, description) VALUES(?, ?)',
         [('Glycolysis', 'A cycle that makes ATP'),
          ('Pentose Phosphate', 'Generates NADPH'),
          ('TCA', 'Inside mitochondria and results in ATP')])  
conn.commit()
#c.execute('SELECT * FROM pathway')

In [50]:
# fetch data for genes table and enzymes table
rows = []
namelist = []

for i in range(len(idlist)):
    print(i)
    handle = Entrez.efetch(db="gene", id = idlist[i], rettype = 'gb', retmode = 'text', retnum=1)
    #print(handle.read())
    if i < 25:
        for line in handle.readlines():
            if 'Name' in line:
                name = line[line.find('Name')+len('Name:')+1:line.find('[')]
                organism = line[line.find('[')+1:line.find(']')]
                print(name)
                print(organism)
            if 'Annotation' in line:
                anno = line[line.find('Annotation')+len('Annotation:')+1:-1]
                annoList = line.split()
                chromosome = annoList[2]
                print(annoList)
                startEndSplit = annoList[4].split('..')
                print(startEndSplit)
                start = (startEndSplit[0])[1:-1]
                end = (startEndSplit[1])[0:-1]
                print(int(start), int(end))
            if 'ID' in line:
                iD = int(line.split(' ')[-1])
                print(iD)
    else:
        for line in handle.readlines():
            if 'Escherichia coli' in line:
                name = line[:line.find('[')]
                organism = 'Escherichia coli'
                print(name)
                print(organism)
            if 'Annotation' in line:
                anno = line[line.find('Annotation')+len('Annotation:')+1:-1]
                annoList = line.split()
                chromosome = '-'
                print(annoList)
                startEndSplit = annoList[2].split('..')
                print(startEndSplit)
                start = (startEndSplit[0])[1:-1]
                end = (startEndSplit[1])[0:-1]
                print(int(start), int(end))
            if 'ID' in line:
                iD = int(line.split(' ')[-1])
                print(iD)
    
    #handle2 = Entrez.efetch(db="nucleotide", id = idlist2[i], rettype = 'fasta', retmode = 'text', retnum=1)
    #print(handle2.read())

    description = '-'
    sequence = '-'
    rows.append((iD, name, organism, chromosome, start, end, sequence))
    print("------------------------------------------")    

0
hexokinase 1 
Homo sapiens (human)
['Annotation:', 'Chromosome', '10', 'NC_000010.11', '(69269991..69401882)']
['(69269991', '69401882)']
(6926999, 69401882)
3098
------------------------------------------
1
phosphoglucomutase 1 
Homo sapiens (human)
['Annotation:', 'Chromosome', '1', 'NC_000001.11', '(63593276..63660245)']
['(63593276', '63660245)']
(6359327, 63660245)
5236
------------------------------------------
2
fructose-bisphosphatase 1 
Homo sapiens (human)
['Annotation:', 'Chromosome', '9', 'NC_000009.12', '(94603133..94640258,', 'complement)']
['(94603133', '94640258,']
(9460313, 94640258)
2203
------------------------------------------
3
glucose-6-phosphate isomerase 
Homo sapiens (human)
['Annotation:', 'Chromosome', '19', 'NC_000019.10', '(34353330..34402413)']
['(34353330', '34402413)']
(3435333, 34402413)
2821
------------------------------------------
4
glucose-6-phosphate isomerase 
Homo sapiens (human)
['Annotation:', 'Chromosome', '19', 'NC_000019.10', '(34353330.

In [34]:
# insert values into enzymes table
ezrows = []
for i in rows:
    namelist.append(i[1])
for x in range(0, len(namelist)):
    if x < 3: 
        y = 'Glycolysis'
    elif (x > 3) and (x < 7):
        y = 'Pentose Phosphate'
    else:
        y = 'TCA'
    temp = (namelist[x], enzymelist[x%12], y)
    ezrows.append(temp)
c.executemany('INSERT INTO enzymes values (?,?,?)', ezrows)
conn.commit()

"""
c.execute("SELECT * FROM enzymes;")
for te in c.fetchall():
    print(te)
"""

'\nc.execute("SELECT * FROM enzymes;")\nfor te in c.fetchall():\n    print(te)\n'

In [21]:
# insert values into genes table
#print(len(rows))
c.executemany('INSERT INTO genes values (?,?,?,?,?,?,?)', rows)
conn.commit()

In [22]:
# import genes table
c.execute("SELECT * FROM genes;")
# print table
tb = pt.PrettyTable()
tb.field_names = ["gene id", "gene name", "organism", "chromosome", "start", "end", "strand"]
for tu in c.fetchall():
    tb.add_row(tu)
print(tb)

+----------+------------------------------------------------------------------------------------+-------------------------------------+------------+----------+-----------+--------+
| gene id  |                                     gene name                                      |               organism              | chromosome |  start   |    end    | strand |
+----------+------------------------------------------------------------------------------------+-------------------------------------+------------+----------+-----------+--------+
|   3098   |                                   hexokinase 1                                     |         Homo sapiens (human)        |     10     | 6926999  |  69401882 |   -    |
|   5236   |                               phosphoglucomutase 1                                 |         Homo sapiens (human)        |     1      | 6359327  |  63660245 |   -    |
|   2203   |                             fructose-bisphosphatase 1                             

In [23]:
# import enzymes table
c.execute("SELECT * FROM enzymes;")
tb = pt.PrettyTable()
tb.field_names = ["enzyme name", "enzyme number", "pathway"]
# print enzymes table
for tu in c.fetchall():
    tb.add_row(tu)
print(tb)

+------------------------------------------------------------------------------------+---------------+-------------------+
|                                    enzyme name                                     | enzyme number |      pathway      |
+------------------------------------------------------------------------------------+---------------+-------------------+
|                                   hexokinase 1                                     |    2.7.1.1    |     Glycolysis    |
|                               phosphoglucomutase 1                                 |    5.4.2.2    |     Glycolysis    |
|                             fructose-bisphosphatase 1                              |    3.1.3.11   |     Glycolysis    |
|                     glucose-6-phosphatase catalytic subunit 2                      |    3.1.3.9    |        TCA        |
|                           glucose-6-phosphate isomerase                            |    5.3.1.9    | Pentose Phosphate |
|               

In [24]:
# import pathway table
c.execute("SELECT * FROM pathway;")
tb = pt.PrettyTable()
tb.field_names = ["pathway name", "description"]
# print pathway table
for tu in c.fetchall():
    tb.add_row(tu)
print(tb)

+-------------------+----------------------------------------+
|    pathway name   |              description               |
+-------------------+----------------------------------------+
|     Glycolysis    |         A cycle that makes ATP         |
| Pentose Phosphate |            Generates NADPH             |
|        TCA        | Inside mitochondria and results in ATP |
+-------------------+----------------------------------------+


In [14]:
"""
from Bio import Entrez
Entrez.email = 'xinxinmo@berkeley.edu'
handle = Entrez.esearch(db='nucleotide',
                        term='Homo sapiens[ORGN]'+"2.7.1.1",
                        sort='relevance',
                        idtype='acc',
                        retmax=1)
for i in Entrez.read(handle)['IdList']:
    #handle = Entrez.efetch(db='nucleotide', id=i, rettype='fasta', retmode='text')
    handle = Entrez.efetch(db='nucleotide', id=i, rettype='fasta', retmode='text')    
    print(handle.read())
"""

'\nfrom Bio import Entrez\nEntrez.email = \'xinxinmo@berkeley.edu\'\nhandle = Entrez.esearch(db=\'nucleotide\',\n                        term=\'Homo sapiens[ORGN]\'+"2.7.1.1",\n                        sort=\'relevance\',\n                        idtype=\'acc\',\n                        retmax=1)\nfor i in Entrez.read(handle)[\'IdList\']:\n    #handle = Entrez.efetch(db=\'nucleotide\', id=i, rettype=\'fasta\', retmode=\'text\')\n    handle = Entrez.efetch(db=\'nucleotide\', id=i, rettype=\'fasta\', retmode=\'text\')    \n    print(handle.read())\n'